<h2>Read Content From PDf - To be Used For RAG</h2>
Post Reading, store in a file as Datafram, woith each Paragraph/ Page content as a Row

In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# 2. load model
SentenceTransformer = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

f:\GitHub\RAG-Langchain-App-Using-Llama\llama3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
from PyPDF2 import PdfReader
import pandas as pd
reader = PdfReader("sourceData\A Brief History of Time .. Stephen Hawking.pdf")
number_of_pages = len(reader.pages)
textChunksDF =  pd.DataFrame(columns=['TrainingData', 'Embeddings'])

#Read content from all pages, compile them into One Txt File
for page in range(number_of_pages):
    # pageText = reader.pages[page]
    plainText = reader.pages[page].extract_text()
    newRow = {'TrainingData':plainText.replace("\n", ""),'Embeddings': SentenceTransformer.encode(plainText)}
    textChunksDF = textChunksDF._append(newRow, ignore_index=True)

print(textChunksDF)

f:\GitHub\RAG-Langchain-App-Using-Llama\llama3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


                                          TrainingData  \
0    Chapter 1  - Our Picture of the UniverseChapte...   
1    future (or have we?) but I discuss a possible ...   
2    CHAPTER 1OUR PICTURE OF THE UNIVERSE A well-kn...   
3    Figure 1:1The planets themselves moved on smal...   
4    Kepler, and the Italian, Galileo Galilei – sta...   
..                                                 ...   
96   GLOSSARY Absolute zero : The lowest possible t...   
97   Elementary particle : A particle that, it is b...   
98   force.Particle accelerator : A machine that, u...   
99   Stationary state : One that is not changing wi...   
100  ACKNOWLEDGMENTS Many people have helped me in ...   

                                            Embeddings  
0    [0.120179795, 0.15168652, 0.41615, -0.82843953...  
1    [0.029344482, 0.0025265208, 0.46344772, -0.714...  
2    [0.20948802, 0.2716869, 0.5334739, -0.9225296,...  
3    [0.18583146, 0.3157446, 0.78558713, -1.42911, ...  
4    [0.103784785,

In [5]:
from datasets import Dataset
# import faiss.cpu as faiss

data = Dataset.from_pandas(textChunksDF)
data = data.add_faiss_index("Embeddings")


  0%|          | 0/1 [00:00<?, ?it/s]